In [1]:
import findspark
findspark.init()

In [2]:
import pyspark 
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("StudyApp").getOrCreate()

In [18]:
df1=spark.read.option("header","true").csv("D:\Items.csv",header=True,inferSchema=True)
df1.show()
df2=spark.read.option("header","true").csv("D:\supplier.csv",header=True,inferSchema=True)
df2.show()

+------+--------+--------+----+-----+----+
|itemid|itemname|itemcost|s_id|grade| qty|
+------+--------+--------+----+-----+----+
|     1|  Pencil|       4|   Y|    B|   6|
|     2|     Pen|      43|   X|    A|   1|
|     3|  Duster|      54|   Y|    C|   3|
|     4|   Chock|      65|   X|    A|   4|
|     5|notebook|    null|null| null|null|
|     7| shopner|    null|   Y|    C|null|
|     8|  Duster|      54|   Y|    C|   3|
|     9|   Chock|      65|   X|    A|   4|
+------+--------+--------+----+-----+----+

+----+------+------+
|s_id|s_name| icost|
+----+------+------+
|   Y|    AD|  1000|
|   X|    ds| 20000|
|   Y|    fd|100000|
|   X|  fgvf|  5000|
|null|    ds| 64000|
|   Y|     i|  4544|
|   Y|     v|   123|
|   X|    gg|  null|
+----+------+------+



In [4]:
df1.createOrReplaceTempView("tempview1")
df2.createOrReplaceTempView("tempview2")

In [5]:
#1.show item details and Supplier details supplied by MR.X
result=spark.sql("select * from tempview1  where s_id=='X' ")
result.show()

+------+--------+--------+----+-----+---+
|itemid|itemname|itemcost|s_id|grade|qty|
+------+--------+--------+----+-----+---+
|     2|     Pen|      43|   X|    A|  1|
|     4|   Chock|      65|   X|    A|  4|
|     9|   Chock|      65|   X|    A|  4|
+------+--------+--------+----+-----+---+



In [19]:
#2.show supplier details who has supplied items with cost>1000
pdf=df1.join(df2,df1.s_id==df2.s_id,'inner')
pdf.createOrReplaceTempView("tempview")
res=spark.sql("select * from tempview  where icost >=1000 ")
res.show()


+------+--------+--------+----+-----+----+----+------+------+
|itemid|itemname|itemcost|s_id|grade| qty|s_id|s_name| icost|
+------+--------+--------+----+-----+----+----+------+------+
|     1|  Pencil|       4|   Y|    B|   6|   Y|     i|  4544|
|     1|  Pencil|       4|   Y|    B|   6|   Y|    fd|100000|
|     1|  Pencil|       4|   Y|    B|   6|   Y|    AD|  1000|
|     2|     Pen|      43|   X|    A|   1|   X|  fgvf|  5000|
|     2|     Pen|      43|   X|    A|   1|   X|    ds| 20000|
|     3|  Duster|      54|   Y|    C|   3|   Y|     i|  4544|
|     3|  Duster|      54|   Y|    C|   3|   Y|    fd|100000|
|     3|  Duster|      54|   Y|    C|   3|   Y|    AD|  1000|
|     4|   Chock|      65|   X|    A|   4|   X|  fgvf|  5000|
|     4|   Chock|      65|   X|    A|   4|   X|    ds| 20000|
|     7| shopner|    null|   Y|    C|null|   Y|     i|  4544|
|     7| shopner|    null|   Y|    C|null|   Y|    fd|100000|
|     7| shopner|    null|   Y|    C|null|   Y|    AD|  1000|
|     8|

In [8]:
#3.show all item details whos supplier details are not available
pdf=df1.join(df2,df1.s_id==df2.s_id,"leftanti")
pdf.toPandas()

,itemid,itemname,itemcost,s_id,grade,qty
0,5,notebook,NaN,None,None,NaN


In [9]:
#4.show item details whose supplier details are available
pdf=df1.join(df2,df1.s_id==df2.s_id,"leftsemi")
pdf.toPandas()
  

,itemid,itemname,itemcost,s_id,grade,qty
0,1,Pencil,4.0,Y,B,6.0
1,2,Pen,43.0,X,A,1.0
2,3,Duster,54.0,Y,C,3.0
3,4,Chock,65.0,X,A,4.0
4,7,shopner,NaN,Y,C,NaN
5,8,Duster,54.0,Y,C,3.0
6,9,Chock,65.0,X,A,4.0


In [10]:
#5.show item details along with supplier details for such items ,for which supplier details are  available and item name starts with 'b'
result=spark.sql("select s_name from tempview2 where s_name LIKE 'i%'")
result.show()


+------+
|s_name|
+------+
|     i|
+------+



In [21]:
#6.Show supplier wise number of items supplied, sum ,min ,max total of itemcost .
result=spark.sql("select min(icost),max(icost),sum(icost) from tempview")
result.show()


+----------+----------+----------+
|min(icost)|max(icost)|sum(icost)|
+----------+----------+----------+
|       123|    100000|    497668|
+----------+----------+----------+



In [12]:
#7.join overall items available in 2 stores
pdf=df1.join(df2,df1.s_id==df2.s_id,"inner")
pdf.show()

+------+--------+--------+----+-----+----+----+------+--------+
|itemid|itemname|itemcost|s_id|grade| qty|s_id|s_name|itemcost|
+------+--------+--------+----+-----+----+----+------+--------+
|     1|  Pencil|       4|   Y|    B|   6|   Y|     v|     123|
|     1|  Pencil|       4|   Y|    B|   6|   Y|     i|    4544|
|     1|  Pencil|       4|   Y|    B|   6|   Y|    fd|  100000|
|     1|  Pencil|       4|   Y|    B|   6|   Y|    AD|    1000|
|     2|     Pen|      43|   X|    A|   1|   X|    gg|    null|
|     2|     Pen|      43|   X|    A|   1|   X|  fgvf|    5000|
|     2|     Pen|      43|   X|    A|   1|   X|    ds|   20000|
|     3|  Duster|      54|   Y|    C|   3|   Y|     v|     123|
|     3|  Duster|      54|   Y|    C|   3|   Y|     i|    4544|
|     3|  Duster|      54|   Y|    C|   3|   Y|    fd|  100000|
|     3|  Duster|      54|   Y|    C|   3|   Y|    AD|    1000|
|     4|   Chock|      65|   X|    A|   4|   X|    gg|    null|
|     4|   Chock|      65|   X|    A|   